# Chroma

[Chroma](https://docs.trychroma.com/getting-started) 是一个面向开发人员效率和体验的 AI 原生开源向量数据库。Chroma 采用 Apache 2.0 许可协议。

本指南提供了使用 Chroma [`向量存储`](/docs/concepts/#vectorstores) 的快速入门概述。如需了解所有 `Chroma` 功能和配置的详细文档，请访问 [API 参考文档](https://api.js.langchain.com/classes/langchain_community_vectorstores_chroma.Chroma.html)。

:::info Chroma Cloud

Chroma Cloud 提供无服务器的向量和全文搜索服务。它速度极快、成本低廉、可扩展性强且使用便捷。只需 30 秒即可创建一个数据库并体验，注册即可获得 5 美元的免费额度。

[开始使用 Chroma Cloud](https://trychroma.com/signup)

:::

## 概览

### 集成详情

| 类 | 包 | [Python 支持](https://python.langchain.com/docs/integrations/vectorstores/chroma/) | 包的最新版本 |
| :--- | :--- | :---: | :---: |
| [`Chroma`](https://api.js.langchain.com/classes/langchain_community_vectorstores_chroma.Chroma.html) | [`@langchain/community`](https://www.npmjs.com/package/@langchain/community) | ✅ |  ![NPM - 版本](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

## 安装

要使用 Chroma 向量存储，你需要安装 `@langchain/community` 集成包，并将 [Chroma JS SDK](https://www.npmjs.com/package/chromadb) 作为对等依赖项安装。

本指南还将使用 [OpenAI 嵌入](/docs/integrations/text_embedding/openai)，这要求你安装 `@langchain/openai` 集成包。如果你愿意，也可以使用 [其他支持的嵌入模型](/docs/integrations/text_embedding)。

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/openai @langchain/core chromadb
</Npm2Yarn>
```

如果你想在本地运行 Chroma，可以使用 Chroma CLI [运行本地 Chroma 服务器](https://docs.trychroma.com/docs/cli/run)，该 CLI 随 `chromadb` 包一起提供：

```
chroma run
```

你也可以使用官方 Chroma 镜像在 Docker 中运行服务器：

```
docker pull chromadb/chroma 
docker run -p 8000:8000 chromadb/chroma
```

### 凭据

如果你在本地运行 Chroma，则无需提供任何凭据。

如果你是 [Chroma Cloud](https://trychroma.com/signup) 用户，请设置你的 `CHROMA_TENANT`、`CHROMA_DATABASE` 和 `CHROMA_API_KEY` 环境变量。

Chroma CLI 可以为你设置这些变量。首先，通过 CLI [登录](https://docs.trychroma.com/docs/cli/login)，然后使用 [`connect` 命令](https://docs.trychroma.com/docs/cli/db)：

```
chroma db connect [db_name] --env-file
```

如果本指南中你要使用 OpenAI 嵌入，还需要设置你的 OpenAI 密钥：

```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```

如果你希望对模型调用进行自动化跟踪，也可以通过取消注释以下内容来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

```typescript
// process.env.LANGSMITH_TRACING="true"
// process.env.LANGSMITH_API_KEY="your-api-key"
```

## 实例化

### 设置你的嵌入函数

首先，选择你的嵌入函数。这里我们使用 `OpenAIEmbeddings`：

In [ ]:
import { OpenAIEmbeddings } from "@langchain/openai";

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

### 本地运行

一个简单的 `Chroma` 实例化将连接到在 `http://localhost:8000` 上本地运行的 Chroma 服务器：

In [ ]:
import { Chroma } from "@langchain/community/vectorstores/chroma";

const vectorStore = new Chroma(embeddings, {
  collectionName: "a-test-collection"
});

如果你使用不同的配置运行 Chroma 服务器，则可以指定你的 `host`、`port` 以及是否使用 `ssl` 进行连接：

In [ ]:
import { Chroma } from "@langchain/community/vectorstores/chroma";

const vectorStore = new Chroma(embeddings, {
  collectionName: "a-test-collection",
  host: "your-host-address",
  port: 8080
});

### Chroma Cloud

连接 Chroma Cloud 时，请提供你的 `tenant`、`database` 和 `chromaCloudAPIKey`：

In [ ]:
import { Chroma } from "@langchain/community/vectorstores/chroma";

const vectorStore = new Chroma(embeddings, {
  collectionName: "a-test-collection",
  tenant: process.env.CHROMA_TENANT,
  database: process.env.CHROMA_DATABASE,
  chromaCloudAPIKey: process.env.CHROMA_API_KEY
});

## 管理向量存储

### 向向量存储中添加项目

In [ ]:
import type { Document } from "@langchain/core/documents";

const document1: Document = {
  pageContent: "The powerhouse of the cell is the mitochondria",
  metadata: { source: "https://example.com" }
};

const document2: Document = {
  pageContent: "Buildings are made out of brick",
  metadata: { source: "https://example.com" }
};

const document3: Document = {
  pageContent: "Mitochondria are made out of lipids",
  metadata: { source: "https://example.com" }
};

const document4: Document = {
  pageContent: "The 2024 Olympics are in Paris",
  metadata: { source: "https://example.com" }
}

const documents = [document1, document2, document3, document4];

await vectorStore.addDocuments(documents, { ids: ["1", "2", "3", "4"] });

### 从向量存储中删除项目

你可以通过如下方式根据 ID 从 Chroma 中删除文档：

In [ ]:
await vectorStore.delete({ ids: ["4"] });

## 查询向量存储

一旦创建了向量存储并添加了相关文档，您很可能希望在运行链或代理时查询该存储。

### 直接查询

执行一个简单的相似性搜索可以按照以下方式完成：

In [ ]:
const filter = { source: "https://example.com" };

const similaritySearchResults = await vectorStore.similaritySearch("biology", 2, filter);

for (const doc of similaritySearchResults) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}

更多关于Chroma过滤器语法的信息请参见[此页面](https://docs.trychroma.com/guides#filtering-by-metadata)。

如果你想执行相似性搜索并获取相应的分数，可以运行：

In [ ]:
const similaritySearchWithScoreResults = await vectorStore.similaritySearchWithScore("biology", 2, filter)

for (const [doc, score] of similaritySearchWithScoreResults) {
  console.log(`* [SIM=${score.toFixed(3)}] ${doc.pageContent} [${JSON.stringify(doc.metadata)}]`);
}

### 通过转换为检索器进行查询

你还可以将向量存储转换为[检索器](/docs/concepts/retrievers)，以便在你的链中更方便地使用。

In [ ]:
const retriever = vectorStore.asRetriever({
  // Optional filter
  filter: filter,
  k: 2,
});
await retriever.invoke("biology");

### 检索增强生成的用法

有关如何将此向量存储用于检索增强生成（RAG）的指南，请参阅以下部分：

- [教程：使用外部知识](/docs/tutorials/#working-with-external-knowledge)。
- [操作指南：使用RAG进行问答](/docs/how_to/#qa-with-rag)
- [检索概念文档](/docs/concepts/retrieval)

## API 参考文档

如需详细了解所有 `Chroma` 特性和配置，请前往 [API 参考文档](https://api.js.langchain.com/classes/langchain_community_vectorstores_chroma.Chroma.html)